In [1]:
import findspark
findspark.init()

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .appName("MyApp") \
    .master("local[*]") \
    .getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/06/14 16:56:11 WARN Utils: Your hostname, kirans-mac.local, resolves to a loopback address: 127.0.0.1; using 172.18.197.149 instead (on interface en0)
25/06/14 16:56:11 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/06/14 16:56:11 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/06/14 16:56:11 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


# Create Dataframe

In [3]:
import datetime
from pyspark.sql import Row

users = [{'id':1,
        'first_name':'kiran',
        'last_name':'chinta',
         'email':'k.c@gmail.com',
         'phone_numbers':Row(mobile=123,home=456),
         'courses':[1,2],
         'is_customer':True,
         'amount_paid':100.55,
         'customer_from':datetime.date(2012,1,1),
        'last_updated_ts':datetime.datetime(2021,2,2,1,2,3)
        },
        {'id':2,
        'first_name':'goats',
        'last_name':'manchi',
         'email':'g.m@gmail.com',
         'phone_numbers':Row(mobile=246,home=680),
         'courses':[11,2],
         'is_customer':False,
         'amount_paid':110.55,
         'customer_from':datetime.date(2012,1,1),
        'last_updated_ts':datetime.datetime(2021,2,2,1,2,3)
        },
        {'id':3,
        'first_name':'ss',
        'last_name':'raj',
         'email':'s.s.r@gmail.com',
         'phone_numbers':Row(mobile=135,home=579),
         'courses':[11,22],
         'is_customer':False,
         'amount_paid':None,
         'customer_from':datetime.date(2012,2,2),
        'last_updated_ts':datetime.datetime(2021,4,5,1,2,3)
        }]

In [4]:
u_df = spark.createDataFrame(users)
u_df.show()

+-----------+--------+-------------+---------------+----------+---+-----------+---------+-------------------+-------------+
|amount_paid| courses|customer_from|          email|first_name| id|is_customer|last_name|    last_updated_ts|phone_numbers|
+-----------+--------+-------------+---------------+----------+---+-----------+---------+-------------------+-------------+
|     100.55|  [1, 2]|   2012-01-01|  k.c@gmail.com|     kiran|  1|       true|   chinta|2021-02-02 01:02:03|   {123, 456}|
|     110.55| [11, 2]|   2012-01-01|  g.m@gmail.com|     goats|  2|      false|   manchi|2021-02-02 01:02:03|   {246, 680}|
|       NULL|[11, 22]|   2012-02-02|s.s.r@gmail.com|        ss|  3|      false|      raj|2021-04-05 01:02:03|   {135, 579}|
+-----------+--------+-------------+---------------+----------+---+-----------+---------+-------------------+-------------+



# drop Duplicates

In [7]:
# complete row should be distinct
u_df.distinct().show()

+-----------+--------+-------------+---------------+----------+---+-----------+---------+-------------------+-------------+
|amount_paid| courses|customer_from|          email|first_name| id|is_customer|last_name|    last_updated_ts|phone_numbers|
+-----------+--------+-------------+---------------+----------+---+-----------+---------+-------------------+-------------+
|     100.55|  [1, 2]|   2012-01-01|  k.c@gmail.com|     kiran|  1|       true|   chinta|2021-02-02 01:02:03|   {123, 456}|
|     110.55| [11, 2]|   2012-01-01|  g.m@gmail.com|     goats|  2|      false|   manchi|2021-02-02 01:02:03|   {246, 680}|
|       NULL|[11, 22]|   2012-02-02|s.s.r@gmail.com|        ss|  3|      false|      raj|2021-04-05 01:02:03|   {135, 579}|
+-----------+--------+-------------+---------------+----------+---+-----------+---------+-------------------+-------------+



In [9]:
# drop duplicates based on a column
u_df.drop_duplicates(['customer_from']).show()

+-----------+--------+-------------+---------------+----------+---+-----------+---------+-------------------+-------------+
|amount_paid| courses|customer_from|          email|first_name| id|is_customer|last_name|    last_updated_ts|phone_numbers|
+-----------+--------+-------------+---------------+----------+---+-----------+---------+-------------------+-------------+
|     100.55|  [1, 2]|   2012-01-01|  k.c@gmail.com|     kiran|  1|       true|   chinta|2021-02-02 01:02:03|   {123, 456}|
|       NULL|[11, 22]|   2012-02-02|s.s.r@gmail.com|        ss|  3|      false|      raj|2021-04-05 01:02:03|   {135, 579}|
+-----------+--------+-------------+---------------+----------+---+-----------+---------+-------------------+-------------+



In [10]:
# drop duplicates based on subset of columns
u_df.drop_duplicates(['customer_from','id']).show()

+-----------+--------+-------------+---------------+----------+---+-----------+---------+-------------------+-------------+
|amount_paid| courses|customer_from|          email|first_name| id|is_customer|last_name|    last_updated_ts|phone_numbers|
+-----------+--------+-------------+---------------+----------+---+-----------+---------+-------------------+-------------+
|     100.55|  [1, 2]|   2012-01-01|  k.c@gmail.com|     kiran|  1|       true|   chinta|2021-02-02 01:02:03|   {123, 456}|
|     110.55| [11, 2]|   2012-01-01|  g.m@gmail.com|     goats|  2|      false|   manchi|2021-02-02 01:02:03|   {246, 680}|
|       NULL|[11, 22]|   2012-02-02|s.s.r@gmail.com|        ss|  3|      false|      raj|2021-04-05 01:02:03|   {135, 579}|
+-----------+--------+-------------+---------------+----------+---+-----------+---------+-------------------+-------------+



# drop null records

In [12]:
# drop if any of the columns is null
u_df.dropna().show()

+-----------+-------+-------------+-------------+----------+---+-----------+---------+-------------------+-------------+
|amount_paid|courses|customer_from|        email|first_name| id|is_customer|last_name|    last_updated_ts|phone_numbers|
+-----------+-------+-------------+-------------+----------+---+-----------+---------+-------------------+-------------+
|     100.55| [1, 2]|   2012-01-01|k.c@gmail.com|     kiran|  1|       true|   chinta|2021-02-02 01:02:03|   {123, 456}|
|     110.55|[11, 2]|   2012-01-01|g.m@gmail.com|     goats|  2|      false|   manchi|2021-02-02 01:02:03|   {246, 680}|
+-----------+-------+-------------+-------------+----------+---+-----------+---------+-------------------+-------------+



In [13]:
# drop if complete row is null
u_df.dropna('all').show()

+-----------+--------+-------------+---------------+----------+---+-----------+---------+-------------------+-------------+
|amount_paid| courses|customer_from|          email|first_name| id|is_customer|last_name|    last_updated_ts|phone_numbers|
+-----------+--------+-------------+---------------+----------+---+-----------+---------+-------------------+-------------+
|     100.55|  [1, 2]|   2012-01-01|  k.c@gmail.com|     kiran|  1|       true|   chinta|2021-02-02 01:02:03|   {123, 456}|
|     110.55| [11, 2]|   2012-01-01|  g.m@gmail.com|     goats|  2|      false|   manchi|2021-02-02 01:02:03|   {246, 680}|
|       NULL|[11, 22]|   2012-02-02|s.s.r@gmail.com|        ss|  3|      false|      raj|2021-04-05 01:02:03|   {135, 579}|
+-----------+--------+-------------+---------------+----------+---+-----------+---------+-------------------+-------------+



In [14]:
# drop the record if both amount_paid and is_customer is null
u_df.dropna('all',subset=['amount_paid','is_customer']).show()

+-----------+--------+-------------+---------------+----------+---+-----------+---------+-------------------+-------------+
|amount_paid| courses|customer_from|          email|first_name| id|is_customer|last_name|    last_updated_ts|phone_numbers|
+-----------+--------+-------------+---------------+----------+---+-----------+---------+-------------------+-------------+
|     100.55|  [1, 2]|   2012-01-01|  k.c@gmail.com|     kiran|  1|       true|   chinta|2021-02-02 01:02:03|   {123, 456}|
|     110.55| [11, 2]|   2012-01-01|  g.m@gmail.com|     goats|  2|      false|   manchi|2021-02-02 01:02:03|   {246, 680}|
|       NULL|[11, 22]|   2012-02-02|s.s.r@gmail.com|        ss|  3|      false|      raj|2021-04-05 01:02:03|   {135, 579}|
+-----------+--------+-------------+---------------+----------+---+-----------+---------+-------------------+-------------+



In [15]:
# drop the record if any of  amount_paid and is_customer is null
u_df.dropna('any',subset=['amount_paid','is_customer']).show()

+-----------+-------+-------------+-------------+----------+---+-----------+---------+-------------------+-------------+
|amount_paid|courses|customer_from|        email|first_name| id|is_customer|last_name|    last_updated_ts|phone_numbers|
+-----------+-------+-------------+-------------+----------+---+-----------+---------+-------------------+-------------+
|     100.55| [1, 2]|   2012-01-01|k.c@gmail.com|     kiran|  1|       true|   chinta|2021-02-02 01:02:03|   {123, 456}|
|     110.55|[11, 2]|   2012-01-01|g.m@gmail.com|     goats|  2|      false|   manchi|2021-02-02 01:02:03|   {246, 680}|
+-----------+-------+-------------+-------------+----------+---+-----------+---------+-------------------+-------------+



In [16]:
spark.stop()